In [ ]:
import numpy as np
import cupy as cp
import dxchange
import matplotlib.pyplot as plt
import cv2
import xraylib

from holotomocupy.holo import G
from holotomocupy.magnification import M
from holotomocupy.shift import S
from holotomocupy.utils import *

%matplotlib inline

np.random.seed(10)

# Init data sizes and parametes of the PXM of ID16A

In [ ]:
n = 2048  # object size in each dimension

noise = 0
z1c = -20e-3
# thickness of the coded aperture
code_thickness = 1.8e-6 #in m
# feature size
ill_feature_size = 2e-6 #in m

# ntheta = int(sys.argv[1])  # number of angles (rotations)
# noise = int(sys.argv[2])#sys.argv[2]=='True'
# z1c = float(sys.argv[3])  # positions of the code and the probe for reconstruction

center = n/2 # rotation axis
npos = 3  # number of code positions
detector_pixelsize = 3e-6
energy = 33.35  # [keV] xray energy
wavelength = 1.2398419840550367e-09/energy  # [m] wave length
focusToDetectorDistance = 1.28  # [m]
sx0 = 3.7e-4
z1 = 4.584e-3-sx0# np.array([4.584e-3, 4.765e-3, 5.488e-3, 6.9895e-3])[:npos]-sx0
z1 = np.tile(z1, [npos])
z2 = focusToDetectorDistance-z1
distances = (z1*z2)/focusToDetectorDistance
magnifications = focusToDetectorDistance/z1
voxelsize = detector_pixelsize/magnifications[0]*2048/n  # object voxel size
# magnification when propagating from the probe plane to the detector
magnifications2 = z1/z1c
distances2 = (z1-z1c)/(z1c/z1)#magnifications2
# allow padding if there are shifts of the probe
pad = n//8
# sample size after demagnification
ne = n+2*pad

show = True

flg = f'{n}_{1}_{npos}_{z1c}_{noise}_code'
path = f'/data2/vnikitin/coded_apertures_2d'

### Generate siemens star and form the transmittance function for it

In [ ]:
img = np.zeros((ne, ne, 3), np.uint8)
triangle = np.array([(ne//16, ne//2-ne//32), (ne//16, ne//2+ne//32), (ne//2-ne//128, ne//2)], np.float32)
star = img[:,:,0]*0
for i in range(0, 360, 15):
    img = np.zeros((ne, ne, 3), np.uint8)
    degree = i
    theta = degree * np.pi / 180
    rot_mat = np.array([[np.cos(theta), -np.sin(theta)],
                        [np.sin(theta), np.cos(theta)]], np.float32)    
    rotated = cv2.gemm(triangle-ne//2, rot_mat, 1, None, 1, flags=cv2.GEMM_2_T)+ne//2
    cv2.fillPoly(img, [np.int32(rotated)], (255, 0, 0))
    star+=img[:,:,0]
[x,y] = np.meshgrid(np.arange(-ne//2,ne//2),np.arange(-ne//2,ne//2))
x = x/ne*2
y = y/ne*2
# add holes in triangles
circ = (x**2+y**2>0.355)+(x**2+y**2<0.345)
circ *= (x**2+y**2>0.083)+(x**2+y**2<0.08)
circ *= (x**2+y**2>0.0085)+(x**2+y**2<0.008)
star = star*circ/255

v = np.arange(-ne//2,ne//2)/ne
[vx,vy] = np.meshgrid(v,v)
v = np.exp(-5*(vx**2+vy**2))
fu = np.fft.fftshift(np.fft.fftn(np.fft.fftshift(star)))
star = np.fft.fftshift(np.fft.ifftn(np.fft.fftshift(fu*v))).real

delta = 1-xraylib.Refractive_Index_Re('Au',energy,19.3)
beta = xraylib.Refractive_Index_Im('Au',energy,19.3)

thickness = 1e-6/voxelsize # siemens star thickness in pixels
# form Transmittance function
u = star*(-delta+1j*beta) # note -delta
Ru = u*thickness 
psi = np.exp(1j * Ru * voxelsize * 2 * np.pi / wavelength)[np.newaxis].astype('complex64')


mshow_polar(psi[0],show)

## Download probe

In [ ]:
!wget -nc https://g-110014.fd635.8443.data.globus.org/holotomocupy/examples_synthetic/data/prb_id16a/prb_abs_2048.tiff -P data/prb_id16a
!wget -nc https://g-110014.fd635.8443.data.globus.org/holotomocupy/examples_synthetic/data/prb_id16a/prb_phase_2048.tiff -P data/prb_id16a

prb_abs = read_tiff(f'data/prb_id16a/prb_abs_2048.tiff')[0:1]
prb_phase = read_tiff(f'data/prb_id16a/prb_phase_2048.tiff')[0:1]
prb = prb_abs*np.exp(1j*prb_phase).astype('complex64')


prb = np.pad(prb[:,pad:-pad,pad:-pad],((0,0),(2*pad,2*pad),(2*pad,2*pad)),'symmetric')#prb[:, 128-pad:-128+pad, 128-pad:-128+pad]
prb /= np.mean(np.abs(prb))

v = np.arange(-(n+2*pad)//2,(n+2*pad)//2)/(n+2*pad)
[vx,vy] = np.meshgrid(v,v)
v=np.exp(-20*(vx**2+vy**2))
prb = np.fft.fftshift(np.fft.fftn(np.fft.fftshift(prb)))
prb = np.fft.fftshift(np.fft.ifftn(np.fft.fftshift(prb*v)))
prb = prb.astype('complex64')

# initial probe
mshow_polar(prb[0],show)
# probe at the code place
prb = G(cp.array(prb),wavelength,voxelsize,-distances2[0])
mshow_polar(prb[0],show)

### Generate a coded aperture, make it twice bigger than the sample to allow motion

In [ ]:
import random
import xraylib
import scipy.ndimage as ndimage



random.seed(10)
nill = 2*ne
ill_global = np.zeros([1,nill,nill],dtype='bool')
for k in  range(ill_global.shape[0]):
    ill0 = np.zeros([nill*nill],dtype='bool')
    ill_ids = random.sample(range(0, nill*nill), int(nill*nill*0.55))
    ill0[ill_ids] = 1
    ill_global[k] = ill0.reshape(nill,nill)

# form codes for simulations
nill = int(ne*voxelsize/np.abs(magnifications2[0])//(ill_feature_size*2))*2
ill = np.zeros([1,nill,nill],dtype='bool')
for k in  range(ill.shape[0]):
    ill0 = ill_global[k]
    ill[k] = ill0[ill0.shape[0]//2-nill//2:ill0.shape[0]//2+(nill)//2,ill0.shape[1]//2-nill//2:ill0.shape[1]//2+(nill)//2]#.reshape(nill,nill)
    
ill = ndimage.zoom(ill,[1,2*ne/nill,2*ne/nill],order=0,grid_mode=True,mode='grid-wrap')

# print(np.sum(ill),2*ne*2*ne//2)

# fsize = int(2*ne//nill)+1
# x = np.linspace(-1,1,fsize)
# [x,y] = np.meshgrid(x,x)
# circ = x**2+y**2<0.9
# ill = ill*0
# while cp.sum(ill)<2*ne*2*ne//2:
#     stx = int(cp.random.randint(ill.shape[1]-fsize))
#     sty = int(cp.random.randint(ill.shape[1]-fsize))
#     # print(stx,sty,ill[:,sty:sty+fsize,stx:stx+fsize].shape)
#     ill[:,sty:sty+fsize,stx:stx+fsize]+=circ
#     if cp.amax(ill[:,sty:sty+fsize,stx:stx+fsize]>1):
#         ill[:,sty:sty+fsize,stx:stx+fsize]-=circ

#     stx = int(cp.random.randint(ill.shape[1]-fsize))
#     sty = int(cp.random.randint(ill.shape[1]-fsize))

# ill = ill*0 + np.random.random(ill.shape)*2

# ill_ids = np.array(ill_ids)
# idsy = ill_ids//(nill)
# idsx = ill_ids%(nill)
# ill[:]=0

# ill_ids = random.sample(range(0, 2*ne*2*ne), int(2*ne*2*ne/2))
# ill0 = np.zeros([2*ne*2*ne],dtype='bool')
# ill0[ill_ids] = 1
# ill[k] = ill0.reshape(2*ne,2*ne)

# for k in range(len(ill_ids)):
#     if idsy[k]*fsize+fsize<=ill.shape[1] and idsx[k]*fsize+fsize<=ill.shape[1]:
#         ill[:,idsy[k]*fsize:idsy[k]*fsize+fsize,idsx[k]*fsize:idsx[k]*fsize+fsize]=circ
# print(np.sum(ill),2*ne*2*ne//2)


# print(ill.shape)
# for k in range(ill.shape[1]//fsize):
#     for j in range(ill.shape[2]//fsize):
#         ill[0,k*fsize:(k+1)*fsize,j*fsize:(j+1)*fsize]*=circ

# print(nill)

delta = 1-xraylib.Refractive_Index_Re('Au',energy,19.3)
beta = xraylib.Refractive_Index_Im('Au',energy,19.3)
print(beta,delta,delta/beta)

# delta = 1-xraylib.Refractive_Index_Re('C21H25ClO',energy,1.19)
# beta = xraylib.Refractive_Index_Im('C21H25ClO',energy,1.19)

print(beta,delta,delta/beta)
thickness = code_thickness/voxelsize # thickness in pixels


# form Transmittance function

Rill = ill*(-delta+1j*beta)*thickness 
Rill=ndimage.rotate(Rill, 45, axes=(2, 1), reshape=False, order=3, mode='reflect',
                   prefilter=True)

v = np.arange(-2*ne//2,2*ne//2)/2/ne
[vx,vy] = np.meshgrid(v,v)
v = np.exp(-10*(vx**2+vy**2))
fill = np.fft.fftshift(np.fft.fftn(np.fft.fftshift(Rill)))
Rill = np.fft.fftshift(np.fft.ifftn(np.fft.fftshift(fill*v)))
Rill = Rill.astype('complex64')

code = np.exp(1j * Rill * voxelsize * 2 * np.pi / wavelength).astype('complex64')

# code=code*0+1
mshow_polar(code[0],show)

mshow_polar(code[0,:n//4,:n//4],show)


### Shifts/drifts

In [ ]:
# shifts of codes
shifts_code = np.round((np.random.random([1, npos, 2]).astype('float32')-0.5)*ne/2)

### Compute holographic projections for all positions

In [ ]:
def _fwd_holo(psi, shifts_code, code, prb):
    prb = cp.array(prb)
    code = cp.array(code)

    data = cp.zeros([psi.shape[0], npos, n, n], dtype='complex64')
    for i in range(npos):
        psir = psi.copy()
        prbr = cp.tile(prb, [psi.shape[0], 1, 1])
        coder = cp.tile(code, [psi.shape[0], 1, 1])
        
        # shift and crop the code 
        coder = S(coder, shifts_code[:, i])
        coder = coder[:, ne-n//2-pad:ne+n//2+pad, ne-n//2-pad:ne+n//2+pad]
        coder *= prbr
        
        # propagate the code to the probe plane
        coder = G(coder, wavelength, voxelsize, distances2[i])
        
        # multiply the ill code and object
        psir *= (coder)                
        # propagate all to the detector
        psir = G(psir, wavelength, voxelsize, distances[i])
        # unpad
        data[:, i] = psir[:, pad:n+pad, pad:n+pad]
    return data

def fwd_holo(psi, prb):
    return _fwd_holo(psi, shifts_code, code, prb)

def _fwd_holo0(prb):    
    data = cp.zeros([1,npos, n, n], dtype='complex64')
    for j in range(npos):
        # propagate illumination 
        data[:,j] = G(prb, wavelength, voxelsize, distances[0]+distances2[0])[:,pad:n+pad,pad:n+pad]        
    return data

def fwd_holo0(prb): 
    return _fwd_holo0(prb)

# Apply the forward transform
psi = cp.array(psi)
prb = cp.array(prb)
code = cp.array(code)
shifts_code = cp.array(shifts_code)
fpsi = fwd_holo(psi, prb)
fref = fwd_holo0(prb)

### Take squared absolute value to simulate data on the detector

In [ ]:
data = np.abs(fpsi)**2
ref = np.abs(fref)**2
mshow(data[0,0],show,vmax=3)
mshow(ref[0,0],show,vmax=3)
if noise>0:
    data_noise = cp.random.poisson(data*noise).astype('float32')/noise
    ref_noise = cp.random.poisson(ref*noise).astype('float32')/noise
    mshow(data[0,0]-data_noise[0,0],show)
    data=data_noise
    ref=ref_noise

mshow(data[0,0]/ref[0,0],show,vmax=2)

### Save data and shifts

In [ ]:
print(f'{path}/data/data_{k}_{flg}.tiff')
for k in range(npos):
    write_tiff(data[:,k],f'{path}/data/data_{k}_{flg}')
for k in range(npos):
    write_tiff(ref[:,k],f'{path}/data/ref_{k}_{flg}')
np.save(f'{path}/data/code_{flg}',code)
np.save(f'{path}/data/shifts_code_{flg}', shifts_code)
np.save(f'{path}/data/prb_{flg}', prb)
